In [1]:
# Airrb project rentals predictions

import pandas as pd
import numpy as np

seadata = pd.read_csv('/content/drive/MyDrive/colab/listings.csv.gz')
print(seadata.shape)
print(seadata.info())
seadf = seadata[["accommodates", "bedrooms","bathrooms", "beds", "price","minimum_nights", "maximum_nights", "number_of_reviews"]]
print(seadf.info(), "\n", "\n")





(5904, 75)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5904 entries, 0 to 5903
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5904 non-null   int64  
 1   listing_url                                   5904 non-null   object 
 2   scrape_id                                     5904 non-null   int64  
 3   last_scraped                                  5904 non-null   object 
 4   source                                        5904 non-null   object 
 5   name                                          5904 non-null   object 
 6   description                                   5894 non-null   object 
 7   neighborhood_overview                         4337 non-null   object 
 8   picture_url                                   5904 non-null   object 
 9   host_id                                       5904 n

In [2]:
# For our purposes at the moment, let’s say the living space that we want 
# to rent can accommodate three people.
# we’ll first calculate the distance, using just the accommodates feature, 
# between the first living space in the data set and our own. 
# We can use the NumPy function np.abs() to get the absolute value.

our_acc_value = 3

first_living_space_value = seadf.loc[1,'accommodates']

print(first_living_space_value)

first_distance = np.abs(first_living_space_value - our_acc_value)

print(first_distance, "\n", "\n")

5
2 
 



In [3]:
# The smallest possible Euclidian distance is zero,
#  which would mean the observation we are comparing to is identical to ours,
#   so the result we’ve gotten here makes sense. 
#   In isolation, however, the value doesn’t mean much unless we know how it compares to other values.
#    Let’s calculate the Euclidean distance for each observation in our data set,
#  and look at the range of values we have using pd.value_counts().


seadf['distance'] = np.abs(seadf.accommodates - our_acc_value)
seadf.distance.value_counts().sort_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


0      482
1     3597
2      521
3      667
4       89
5      268
6       38
7       96
8       26
9       55
10       7
11      21
12       8
13      29
Name: distance, dtype: int64

In [4]:
# If we just used the first five values with a distance of 0,
#  our predictions would be biased to the existing ordering of the data set.
#   Instead, we’ll randomize the ordering of the observations and then select 
#   the first five rows with a distance of 0.
#    We’re going to use DataFrame.sample() to randomize the rows.
#     This method is usually used to select a random fraction of the DataFrame,
#      but we’ll tell it to randomly select 100%, which will randomly shuffle the rows for us.

# We’ll also use the random_state parameter which just gives us a reproducible 
# random order so anyone can follow along and get the exact same results.

seadf = seadf.sample(frac=1,random_state=42)
seadf = seadf.sort_values('distance')
seadf.price.head()

4271    $134.00
5477     $95.00
3166    $156.00
823      $96.00
2651    $233.00
Name: price, dtype: object

In [5]:
print(type(seadf['price']))

<class 'pandas.core.series.Series'>


In [6]:
# Before we can take the average of our prices,
#  you’ll notice that our price column has the object type,
#   due to the fact that the prices have dollar signs and commas
#    (our sample above doesn’t show the commas because all the values are less than $1000).

# Let’s clean this column by removing these characters and converting it to a float type,
#  before calculating the mean of the first five values.
#   We’ll use pandas’s Series.str.replace() to remove the stray characters and pass 
#   the regular expression \$|, which will match $ or ,.

seadf['price'] = seadf['price'].astype(str)

seadf['price'] = seadf.price.str.replace("\$|,",'').astype(float)
mean_price = seadf.price.iloc[:5].mean()
mean_price

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


142.8

In [7]:
print(seadf.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5904 entries, 4271 to 5890
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   accommodates       5904 non-null   int64  
 1   bedrooms           5206 non-null   float64
 2   bathrooms          0 non-null      float64
 3   beds               5837 non-null   float64
 4   price              5904 non-null   float64
 5   minimum_nights     5904 non-null   int64  
 6   maximum_nights     5904 non-null   int64  
 7   number_of_reviews  5904 non-null   int64  
 8   distance           5904 non-null   int64  
dtypes: float64(4), int64(5)
memory usage: 461.2 KB
None


In [8]:
# A simple way to test the quality of your model is to:

#     Split the data set into 2 partitions:
#         A training set: contains the majority of the rows (75%)
#         A test set: contains the remaining minority of the rows (25%)
#     Use the rows in the training set to predict the price value for the rows in the test set
#     Compare the predicted values with the actual price values in the test set to see how accurate 
#     the predicted values were.

# We’re going to follow this approach and split the 3,723 rows of our data set into two parts:
#  train_df and test_df in a 75%-25% split.

# train_test_split

# Splitting into train and test dataframes

# We’ll also remove the column we added earlier when we created our first model.

seadf.drop('distance',axis=1)
train_df = seadf.copy().iloc[:2792]
test_df = seadf.copy().iloc[2792:]

In [11]:
# To make things easier for ourselves while we look at metrics,
#  we’ll combine the model we made earlier into a function. 
#  We won’t need to worry about randomizing the rows, 
#  since they’re still randomized from earlier.

def predict_price(new_listing_value,feature_column):
    temp_df = train_df
    temp_df['distance'] = np.abs(seadf[feature_column] - new_listing_value)
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    #print("knn>>>",knn_5)
    predicted_price = knn_5.mean()
    #print("pred>>", predicted_price,"\n")
    return(predicted_price)

# We can now use this function to predict values for our test dataset using the accommodates column.

test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column='accommodates')


In [12]:
# RMSE

test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)
rmse

156.69389956840607

In [13]:
for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:
    test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column=feature)
    test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
    mse = test_df['squared_error'].mean()
    rmse = mse ** (1/2)
    print("RMSE for the {} column: {}".format(feature,rmse))

RMSE for the accommodates column: 156.69389956840607
RMSE for the bedrooms column: 154.53276480558284
RMSE for the bathrooms column: 176.79463588478853
RMSE for the number_of_reviews column: 171.71286489122377
